In [1]:
# Import needed libraries
%reload_ext tensorboard

import datetime

# Data containers
import numpy as np
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Manipulation and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# NLP
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import SnowballStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

import gensim
from gensim.models import Word2Vec

# Machine Learning Models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.naive_bayes import MultinomialNB

# Deep_Learning
import tensorflow as tf
from tensorflow import keras

# This code snippet forces tensorflow to not automatically allocate all GPU ram which can be an issue in notebook environment
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
tf.debugging.disable_traceback_filtering()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Braden\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Braden\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Braden\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Braden\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1 Physical GPUs, 1 Logical GPUs


### Define the Custom Layer for multiple activation functions per layer

In [2]:
# Define custom Layer
class MultiActivationLayer(keras.layers.Layer):
    '''
    Multiple Activation Layer
    
    A neural network layer in which every node has a different activation function applied
    '''
    
    def __init__(self, out_features, activations, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features
        self.activations = activations
    
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
    
    @tf.function
    def call(self, inputs):
        z = tf.matmul(inputs, self.w) + self.b
        shape = tf.shape(z)[0]
        
        # Apply activation function to ouput features from nodes (columns) separately with different activation functions
        #, reshape to 2-D array and concatenate the results from each node in the same order
        nodes = [tf.reshape(self.activations[i%len(self.activations)](z[:,i]), (shape, 1)) for i in range(self.out_features)]
        z = tf.concat(nodes, 1)
        return z
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "out_features": self.out_features,
            "activations": self.activations,
        })
        return config

## Define method for creating the testing models with equivalent architectures

In [3]:
# Define custom architecture using the new layer
def create_model(activations, option='multi', num_classes=1, dropout=False, dropout_rate=0.2, task='classification'):
    if option == 'uniform':
        if not dropout:
            if task == 'classification':
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[0], name='layers_dense_2'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[0], name='layers_dense_3'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[0], name='layers_dense_4'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[0], name='layers_dense_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                 return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[0], name='layers_dense_2'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[0], name='layers_dense_3'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[0], name='layers_dense_4'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[0], name='layers_dense_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
        else:
            if task == 'classification':
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[0], name='layers_dense_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[0], name='layers_dense_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[0], name='layers_dense_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[0], name='layers_dense_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[0], name='layers_dense_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[0], name='layers_dense_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[0], name='layers_dense_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[0], name='layers_dense_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
    elif option == 'multi':
        if not dropout:
            if task == 'classification':
                return keras.models.Sequential([
                    MultiActivationLayer(25, activations, name='layers_multi_1'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(20, activations, name='layers_multi_2'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(15, activations, name='layers_multi_3'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(10, activations, name='layers_multi_4'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(5, activations, name='layers_multi_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                 return keras.models.Sequential([
                    MultiActivationLayer(25, activations, name='layers_multi_1'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(20, activations, name='layers_multi_2'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(15, activations, name='layers_multi_3'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(10, activations, name='layers_multi_4'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(5, activations, name='layers_multi_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
        else:
            if task == 'classification':
                return keras.models.Sequential([
                    MultiActivationLayer(25, activations, name='layers_multi_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(20, activations, name='layers_multi_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(15, activations, name='layers_multi_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(10, activations, name='layers_multi_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(5, activations, name='layers_multi_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                return keras.models.Sequential([
                    MultiActivationLayer(25, activations, name='layers_multi_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(20, activations, name='layers_multi_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(15, activations, name='layers_multi_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(10, activations, name='layers_multi_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(5, activations, name='layers_multi_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
    elif option == 'sequential':
        if len(activations) < 5: raise RuntimeError()
        if not dropout:
            if task == 'classification':
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[1], name='layers_dense_2'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[2], name='layers_dense_3'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[3], name='layers_dense_4'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[4], name='layers_dense_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                 return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[1], name='layers_dense_2'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[2], name='layers_dense_3'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[3], name='layers_dense_4'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[4], name='layers_dense_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
        else: 
            if task == 'classification':
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[1], name='layers_dense_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[2], name='layers_dense_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[3], name='layers_dense_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[4], name='layers_dense_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[1], name='layers_dense_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[2], name='layers_dense_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[3], name='layers_dense_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[4], name='layers_dense_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
    else:
        raise RuntimeError()

## Define method for training all testing models on the data

In [12]:
# Define testing function for generating and running tests
def test(X, y, num_classes=None, X_test=None, y_test=None, task='classification', epochs=100, batch_size=32, task_name=''):
    
    MANN, MANN_drop, UANN1, UANN2, UANN3, UANN4, UANN5, SANN = None, None, None, None, None, None, None, None
    
    if X_test is None or y_test is None:
        # Split data into training and testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    else:
        X_train, y_train = X, y

    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    train_dataset = train_dataset.shuffle(1000).batch(batch_size)
    test_dataset = test_dataset.batch(batch_size)
        
    
    if task == 'classification':
        if num_classes is None:
            raise RuntimeError()
    
        # Create our model
        MANN = create_model(activations=[tf.nn.sigmoid, tf.nn.tanh, tf.nn.leaky_relu, tf.nn.elu, tf.nn.swish], dropout=False, num_classes=num_classes, task=task)
        MANN.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])

        # Create our model with dropout
        MANN_drop = create_model(activations=[tf.nn.sigmoid, tf.nn.tanh, tf.nn.leaky_relu, tf.nn.elu, tf.nn.swish], dropout=True, num_classes=num_classes, task=task)
        MANN_drop.compile(optimizer=keras.optimizers.legacy.Nadam(),
                         loss=keras.losses.CategoricalCrossentropy(),
                         metrics=[keras.metrics.CategoricalAccuracy()])

        # Create uniform model for each activation function
        UANN1 = create_model(activations=[tf.nn.sigmoid], option='uniform', num_classes=num_classes, task=task)
        UANN1.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        UANN2 = create_model(activations=[tf.nn.tanh], option='uniform', num_classes=num_classes, task=task)
        UANN2.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        UANN3 = create_model(activations=[tf.nn.leaky_relu], option='uniform', num_classes=num_classes, task=task)
        UANN3.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        UANN4 = create_model(activations=[tf.nn.elu], option='uniform', num_classes=num_classes, task=task)
        UANN4.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        UANN5 = create_model(activations=[tf.nn.swish], option='uniform', num_classes=num_classes, task=task)
        UANN5.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        # Create sequential activation network
        SANN = create_model(activations=[tf.nn.swish, tf.nn.elu, tf.nn.leaky_relu, tf.nn.tanh, tf.nn.sigmoid], option='sequential', dropout=False, num_classes=num_classes, task=task)
        SANN.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
    elif task == 'regression':
        # Create our model
        MANN = create_model(activations=[tf.nn.sigmoid, tf.nn.tanh, tf.nn.leaky_relu, tf.nn.elu, tf.nn.swish], dropout=False, task=task)
        MANN.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.MeanSquaredError())

        # Create our model with dropout
        MANN_drop = create_model(activations=[tf.nn.sigmoid, tf.nn.tanh, tf.nn.leaky_relu, tf.nn.elu, tf.nn.swish], dropout=True, task=task)
        MANN_drop.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.MeanSquaredError())
        
        # Create uniform model for each activation function
        UANN1 = create_model(activations=[tf.nn.sigmoid], option='uniform', task=task)
        UANN1.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.MeanSquaredError())
        
        UANN2 = create_model(activations=[tf.nn.tanh], option='uniform', task=task)
        UANN2.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.MeanSquaredError())
        
        UANN3 = create_model(activations=[tf.nn.leaky_relu], option='uniform', task=task)
        UANN3.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.MeanSquaredError())
        
        UANN4 = create_model(activations=[tf.nn.elu], option='uniform', task=task)
        UANN4.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.MeanSquaredError())
        
        UANN5 = create_model(activations=[tf.nn.swish], option='uniform', task=task)
        UANN5.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.MeanSquaredError())
        
        # Create sequential activation network
        SANN = create_model(activations=[tf.nn.swish, tf.nn.elu, tf.nn.leaky_relu, tf.nn.tanh, tf.nn.sigmoid], option='sequential', task=task)
        SANN.compile(optimizer=keras.optimizers.legacy.Nadam(),
                     loss=keras.losses.MeanSquaredError())
    else:
        raise RuntimeError()
    
    # Fit the data to the models for a set number of epochs
    print("Training Multi Activation Neural Network...")
    path=f'{task_name}.MANN.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path,
                                                                    save_weights_only=True,
                                                                    monitor='val_loss',
                                                                    mode='min',
                                                                    save_best_only=True)
    MANN_hist = MANN.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, callbacks=[model_checkpoint_callback])
    print('')

    print("Training Multi Activation Neural Network w/ Dropout...")
    path=f'{task_name}.MANN_dropout.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path,
                                                                    save_weights_only=True,
                                                                    monitor='val_loss',
                                                                    mode='min',
                                                                    save_best_only=True)
    MANN_drop_hist = MANN_drop.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, callbacks=[model_checkpoint_callback])
    print('Finished.')


    print("Training Sigmoid Uniform Activation Neural Network...")
    path=f'{task_name}.Sigmoid.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path,
                                                                    save_weights_only=True,
                                                                    monitor='val_loss',
                                                                    mode='min',
                                                                    save_best_only=True)
    UANN1_hist = UANN1.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, callbacks=[model_checkpoint_callback])
    print('Finished.')

    print("Training Tanh Uniform Activation Neural Network...")
    path=f'{task_name}.Tanh.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path,
                                                                    save_weights_only=True,
                                                                    monitor='val_loss',
                                                                    mode='min',
                                                                    save_best_only=True)
    UANN2_hist = UANN2.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, callbacks=[model_checkpoint_callback])
    print('Finished.')

    print("Training Leaky ReLU Uniform Activation Neural Network...")
    path=f'{task_name}.LeakyReLU.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path,
                                                                    save_weights_only=True,
                                                                    monitor='val_loss',
                                                                    mode='min',
                                                                    save_best_only=True)
    UANN3_hist = UANN3.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, callbacks=[model_checkpoint_callback])
    print('Finished.')

    print("Training ELU Uniform Activation Neural Network...")
    path=f'{task_name}.ELU.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path,
                                                                    save_weights_only=True,
                                                                    monitor='val_loss',
                                                                    mode='min',
                                                                    save_best_only=True)
    UANN4_hist = UANN4.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, callbacks=[model_checkpoint_callback])
    print('Finished.')

    print("Training Swish Uniform Activation Neural Network...")
    path=f'{task_name}.Swish.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path,
                                                                    save_weights_only=True,
                                                                    monitor='val_loss',
                                                                    mode='min',
                                                                    save_best_only=True)
    UANN5_hist = UANN5.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, callbacks=[model_checkpoint_callback])
    print('Finished.')

    print("Training Sequential Activation Neural Network...")
    path=f'{task_name}.Sequential.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path,
                                                                    save_weights_only=True,
                                                                    monitor='val_loss',
                                                                    mode='min',
                                                                    save_best_only=True)
    SANN_hist = SANN.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, callbacks=[model_checkpoint_callback])
    print('Finished.')

    # Create dictionaries to return for models and scores
    models = {
        'MANN': MANN,
        'MANN_Dropout': MANN_drop,
        'Sigmoid': UANN1,
        'Tanh': UANN2,
        'LeakyReLU': UANN3,
        'ELU': UANN4,
        'Swish': UANN5,
        'Sequential': SANN
    }
    
    histories = {
        'MANN': MANN_hist,
        'MANN_dropout': MANN_drop_hist,
        'Sigmoid': UANN1_hist,
        'Tanh': UANN2_hist,
        'LeakyReLU': UANN3_hist,
        'ELU': UANN4_hist,
        'Swish': UANN5_hist,
        'Sequential': SANN_hist
    }

    return models, histories

## Regression task

In [ ]:
# Load the data and split into testing and training for processing
data = pd.read_csv('./Datasets/Regression/train.csv')
features = data.drop('TARGET(PRICE_IN_LACS)', axis=1)
target = data['TARGET(PRICE_IN_LACS)']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
# Process and normalize the data
one_hot = OneHotEncoder(sparse_output=False)

# One-Hot encode categorical columns
X_train.append(pd.DataFrame(data=one_hot.fit_transform(X_train[['POSTED_BY', 'BHK_OR_RK']]), columns=one_hot.get_feature_names_out()))
X_test.append(pd.DataFrame(data=one_hot.transform(X_test[['POSTED_BY', 'BHK_OR_RK']]), columns=one_hot.get_feature_names_out()))

# Drop the transformed columns and unneeded columns
X_train.drop(['POSTED_BY', 'BHK_OR_RK', 'ADDRESS'], axis = 1, inplace=True)
X_test.drop(['POSTED_BY', 'BHK_OR_RK', 'ADDRESS'], axis=1, inplace=True)

# Normalize numerical columns
scaler = StandardScaler()
X_train[['BHK_NO.', 'SQUARE_FT', 'LONGITUDE', 'LATITUDE']]= scaler.fit_transform(X_train[['BHK_NO.', 'SQUARE_FT', 'LONGITUDE', 'LATITUDE']])
X_test[['BHK_NO.', 'SQUARE_FT', 'LONGITUDE', 'LATITUDE']] = scaler.transform(X_test[['BHK_NO.', 'SQUARE_FT', 'LONGITUDE', 'LATITUDE']])

In [ ]:
X_test.head()

In [ ]:
# Create and fit all neural networks for evaluations
models, histories = test(X_train.to_numpy(), y_train.to_numpy(), X_test=X_test.to_numpy(), y_test=y_test.to_numpy(), task='regression', epochs=500, batch_size=128, task_name='Regression')

In [ ]:
# Plot the training loss over the epochs
plt.plot(np.array(range(len(histories['MANN'].history['loss']))), histories['MANN'].history['loss'], label='MANN')
plt.plot(np.array(range(len(histories['MANN_dropout'].history['loss']))), histories['MANN_dropout'].history['loss'], label='MANN_dropout')
plt.plot(np.array(range(len(histories['Sigmoid'].history['loss']))), histories['Sigmoid'].history['loss'], label='Sigmoid')
plt.plot(np.array(range(len(histories['Tanh'].history['loss']))), histories['Tanh'].history['loss'], label='Tanh')
plt.plot(np.array(range(len(histories['LeakyReLU'].history['loss']))), histories['LeakyReLU'].history['loss'], label='Leaky ReLU')
plt.plot(np.array(range(len(histories['ELU'].history['loss']))), histories['ELU'].history['loss'], label='ELU')
plt.plot(np.array(range(len(histories['Swish'].history['loss']))), histories['Swish'].history['loss'], label='Swish')
plt.plot(np.array(range(len(histories['Sequential'].history['loss']))), histories['Sequential'].history['loss'], label='Sequential')
plt.legend()
plt.show()

In [ ]:
# Plot the validation loss over the epochs
plt.plot(np.array(range(len(histories['MANN'].history['loss']))), histories['MANN'].history['val_loss'], label='MANN', linewidth=0.75)
plt.plot(np.array(range(len(histories['MANN_dropout'].history['loss']))), histories['MANN_dropout'].history['val_loss'], label='MANN_dropout', linewidth=0.75)
plt.plot(np.array(range(len(histories['Sigmoid'].history['loss']))), histories['Sigmoid'].history['val_loss'], label='Sigmoid', linewidth=0.75)
plt.plot(np.array(range(len(histories['Tanh'].history['loss']))), histories['Tanh'].history['val_loss'], label='Tanh', linewidth=0.75)
plt.plot(np.array(range(len(histories['LeakyReLU'].history['loss']))), histories['LeakyReLU'].history['val_loss'], label='Leaky ReLU', linewidth=0.75)
plt.plot(np.array(range(len(histories['ELU'].history['loss']))), histories['ELU'].history['val_loss'], label='ELU', linewidth=0.75)
plt.plot(np.array(range(len(histories['Swish'].history['loss']))), histories['Swish'].history['val_loss'], label='Swish', linewidth=0.75)
plt.plot(np.array(range(len(histories['Sequential'].history['loss']))), histories['Sequential'].history['val_loss'], label='Sequential', linewidth=0.75)
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

In [ ]:
# Save training history
train_hist_df = pd.DataFrame()
val_hist_df = pd.DataFrame()
for name, callback in histories.items():
    train_hist_df = pd.concat((train_hist_df, pd.DataFrame(callback.history['loss'], columns=[name])), axis=1, join='inner', ignore_index=True)
    val_hist_df = pd.concat((val_hist_df, pd.DataFrame(callback.history['val_loss'], columns=[name])), axis=1, join='inner', ignore_index=True)
    
train_hist_df.to_csv('Regression.training_hist.csv')
val_hist_df.to_csv('Regression.validation_hist.csv')

In [ ]:
# Load the best performing weights for each model and fit the traditional ML models
for name, model in models.items():
    model.load_weights(f'Regression.{name}.tf')

# Traditional models as a baseline comparison
linreg = LinearRegression()
linreg.fit(X_train, y_train)

svm = LinearSVR(max_iter=10000)
svm.fit(X_train, y_train)

In [ ]:
# Get the testing loss for each model
linreg_loss = keras.losses.mean_squared_error(y_test, linreg.predict(X_test)).numpy()
svm_loss = keras.losses.mean_squared_error(y_test, svm.predict(X_test)).numpy()
MANN_loss = models['MANN'].evaluate(X_test, y_test)
MANN_dropout_loss = models['MANN_Dropout'].evaluate(X_test, y_test)
Sigmoid_loss = models['Sigmoid'].evaluate(X_test, y_test)
Tanh_loss = models['Tanh'].evaluate(X_test, y_test)
LeakyReLU_loss = models['LeakyReLU'].evaluate(X_test, y_test)
ELU_loss = models['ELU'].evaluate(X_test, y_test)
Swish_loss = models['Swish'].evaluate(X_test, y_test)
Sequential_loss = models['Sequential'].evaluate(X_test, y_test)

In [ ]:
# Save testing loss to table and display results
results = pd.DataFrame([linreg_loss, svm_loss, MANN_loss, MANN_dropout_loss, Sigmoid_loss, Tanh_loss, LeakyReLU_loss, ELU_loss, Swish_loss, Sequential_loss],
                      index=['Linear Regression', 'Support Vector Machine', 'MANN', 'MANN w/ Dropout', 'Sigmoid NN', 'Tanh NN', 'Leaky ReLU NN', 'ELU NN', 'Swish NN', 'Sequential NN'],
                      columns=['Mean Squared Error'])
results.sort_values('Mean Squared Error', inplace=True)
results.head(10)

## NLP task

In [17]:
# Load the data and split into testing and training for processing
data = pd.read_csv('./Datasets/NLP/twitter_MBTI.csv', index_col=0)
features = data['text']
target = data['label']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42, stratify=target)

In [18]:
data.describe()

,text,label
count,7811,7811
unique,7581,16
top,I just realize that today is dzi's birthday......,infp
freq,2,1282


In [19]:
data.head(100)

,text,label
0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj
1,@Hispanthicckk Being you makes you look cute||...,intj
2,@Alshymi Les balles sont réelles et sont tirée...,intj
3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj
4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj
...,...,...
95,Doing my project on isle neumann|||@fuzy_sox I...,entp
96,she gets it https://t.co/tK0PlXKO2d|||@nakopoc...,entp
97,@WeezingOffline @Max__4__x @raptalksk @Hemant3...,entp
98,Thanks to @About1816 for inviting me to write ...,entp


In [20]:
def wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def lemmatize(document):
    word_pos_tags = nltk.pos_tag(document)
    roots = [lemmatizer.lemmatize(tag[0], wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)]
    return roots

In [21]:
# Process and normalize the data
one_hot = OneHotEncoder(sparse_output=False)
lemmatizer = WordNetLemmatizer()
vectorizer = TfidfVectorizer(max_features=5000)

# One-Hot encode target labels
y_train = pd.DataFrame(data=one_hot.fit_transform(y_train.to_numpy().reshape(-1,1)), columns=one_hot.categories_)
y_test = pd.DataFrame(data=one_hot.transform(y_test.to_numpy().reshape(-1,1)), columns=one_hot.categories_)

# Clean and process text
X_train = X_train.map(lambda x: x.lower().strip())
X_test = X_test.map(lambda x: x.lower().strip())

X_train = X_train.map(lambda x: [token for y in x.split('|||') for token in y.split(' ') if 'http' not in token and '@' not in token])
X_test = X_test.map(lambda x: [token for y in x.split('|||') for token in y.split(' ') if 'http' not in token and '@' not in token])

X_train = X_train.map(lambda x: ' '.join(x))
X_test = X_test.map(lambda x: ' '.join(x))

X_train = X_train.map(word_tokenize)
X_test = X_test.map(word_tokenize)

X_train = X_train.map(lambda x: [word for word in x if word not in stop_words and word.isalpha()])
X_test = X_test.map(lambda x: [word for word in x if word not in stop_words and word.isalpha()])

X_train = X_train.map(lambda x: lemmatize(x))
X_test = X_test.map(lambda x: lemmatize(x))

X_train = X_train.map(lambda x: ' '.join(x))
X_test = X_test.map(lambda x: ' '.join(x))

# Create word embeddings with TF-IDF
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

X_train = np.asarray(X_train.todense())
X_test = np.asarray(X_test.todense())

In [16]:
keras.layers.TextVectorization()

179089

In [11]:
# Create and fit all neural networks for evaluations
models, histories = test(X_train, y_train.to_numpy(), X_test=X_test, y_test=y_test.to_numpy(), num_classes=len(one_hot.categories_[0]), task='classification', epochs=500, batch_size=1, task_name='NLP')

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# Plot the training loss over the epochs
plt.plot(np.array(range(len(histories['MANN'].history['loss']))), histories['MANN'].history['loss'], label='MANN')
plt.plot(np.array(range(len(histories['MANN_dropout'].history['loss']))), histories['MANN_dropout'].history['loss'], label='MANN_dropout')
plt.plot(np.array(range(len(histories['Sigmoid'].history['loss']))), histories['Sigmoid'].history['loss'], label='Sigmoid')
plt.plot(np.array(range(len(histories['Tanh'].history['loss']))), histories['Tanh'].history['loss'], label='Tanh')
plt.plot(np.array(range(len(histories['LeakyReLU'].history['loss']))), histories['LeakyReLU'].history['loss'], label='Leaky ReLU')
plt.plot(np.array(range(len(histories['ELU'].history['loss']))), histories['ELU'].history['loss'], label='ELU')
plt.plot(np.array(range(len(histories['Swish'].history['loss']))), histories['Swish'].history['loss'], label='Swish')
plt.plot(np.array(range(len(histories['Sequential'].history['loss']))), histories['Sequential'].history['loss'], label='Sequential')
plt.legend()
plt.show()

In [ ]:
# Plot the validation loss over the epochs
plt.plot(np.array(range(len(histories['MANN'].history['loss']))), histories['MANN'].history['val_loss'], label='MANN', linewidth=0.75)
plt.plot(np.array(range(len(histories['MANN_dropout'].history['loss']))), histories['MANN_dropout'].history['val_loss'], label='MANN_dropout', linewidth=0.75)
plt.plot(np.array(range(len(histories['Sigmoid'].history['loss']))), histories['Sigmoid'].history['val_loss'], label='Sigmoid', linewidth=0.75)
plt.plot(np.array(range(len(histories['Tanh'].history['loss']))), histories['Tanh'].history['val_loss'], label='Tanh', linewidth=0.75)
plt.plot(np.array(range(len(histories['LeakyReLU'].history['loss']))), histories['LeakyReLU'].history['val_loss'], label='Leaky ReLU', linewidth=0.75)
plt.plot(np.array(range(len(histories['ELU'].history['loss']))), histories['ELU'].history['val_loss'], label='ELU', linewidth=0.75)
plt.plot(np.array(range(len(histories['Swish'].history['loss']))), histories['Swish'].history['val_loss'], label='Swish', linewidth=0.75)
plt.plot(np.array(range(len(histories['Sequential'].history['loss']))), histories['Sequential'].history['val_loss'], label='Sequential', linewidth=0.75)
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

In [ ]:
# Save training history
train_hist_df = pd.DataFrame()
val_hist_df = pd.DataFrame()
for name, callback in histories.items():
    train_hist_df = pd.concat(hist_df, pd.DataFrame(callback.history['loss'], columns=[name]), axis=1, join='inner', ignore_index=True)
    val_hist_df = pd.concat(hist_df, pd.DataFrame(callback.history['val_loss'], columns=[name]), axis=1, join='inner', ignore_index=True)
    
train_hist_df.to_csv('NLP.training_hist.csv')
val_his_df.to_csv('NLP.validation_hist.csv')

In [ ]:
# Load the best performing weights for each model and fit the traditional ML models
for name, model in models.items():
    model.load_weights(f'NLP.{name}.tf')
    
# Traditional Models as a baseline comparison


In [ ]:
# Get performance metrics for each model

In [ ]:
# Save metrics to a table and display results

In [ ]:
# Visual results with confusion matrix